This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [1]:
import time
import numpy as np
import dgl
import networkx as nx
import torch
import torch.nn as nn

from src import utils
from src.baselines_archs import MLP, GAT, GCNN, GraphSAGE, GIN
from src.baselines_models import NodeClassModel, GF_NodeClassModel
from src.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows, Dual_GFGCN, NV_GFGCN

# SEED = 0
N_RUNS = 10
SEED = 10
PATH = 'results/try_datasets'
SAVE = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:0


## Auxiliary functions

In [2]:
def select_GSO(A, gso_type, self_loops=True, norm=False, dev='cpu'):
    if gso_type == 'A-dgl':
        if self_loops:
            S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop()
        else:
            S = dgl.from_networkx(nx.from_numpy_array(A))
        
    elif gso_type == 'A':
        if norm:
            S = torch.Tensor(normalize_gso(A, 'both', add_id=False))
        else:
            S = torch.Tensor(A)
        
    elif gso_type == 'none' or gso_type is None:
        return None
    else:
        raise ValueError('Unknown type of GSO')

    return S.to(dev)

def run_exp(Exps, datasets, n_runs, device):
    accs_test = np.zeros((len(Exps), len(datasets), n_runs))
    accs_max_val = np.zeros_like(accs_test)
    ellapsed_times = np.zeros_like(accs_test)

    for j, dataset in enumerate(datasets):
        print(dataset)

        for i in range(n_runs):
            print(f'{i}:', end=' ')

            A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
            N = A.shape[0]

            # Default arguments
            d_arc_args = {'in_dim': feat.shape[1], 'out_dim': n_class,}

            last_str = ' - '
            for k, exp in enumerate(Exps):
                arc_args = {**d_arc_args, **exp['arc_args']}

                arch = exp['arch'](**arc_args)
                S_sel = select_GSO(A, **exp['sel_GSO'], dev=device)
                model = exp['model'](arch, S_sel, masks=masks, **exp['mod_args'], device=device)

                t_i = time.time()
                loss, acc = model.train(feat, labels, **exp['train_args'])
                ellapsed_t = (time.time()-t_i)/60
            
                accs_test[k,j,i] = model.test(feat, model.S, labels, masks['test'])
                accs_max_val[k,j,i] = acc["test"][np.argmax(acc["val"])]
                ellapsed_times[k,j,i] = ellapsed_t

                if k == len(Exps) - 1:
                    last_str = ''

                print(f'{accs_test[k,j,i]:.3f} ({ellapsed_times[k,j,i]:.3f})', end=last_str)
            
            print()
    
    return accs_test, accs_max_val, ellapsed_times

def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

## Heterophilicc datasets

In [3]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'ChameleonDataset']
ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()


EXPS = [
    ### Our models
    {'arch': GFGCN, 'arc_args': {'diff_layer': GFGCNLayer, 'hid_dim': 32, 'n_layers': 2, 'K': 3, 'act': ACT, 'h0': 1,
     'l_act': LAST_ACT, 'dropout': .25,}, 'sel_GSO': {'gso_type': 'A', 'norm': False}, 'model': GF_NodeClassModel,
     'mod_args': {'loss': LOSS_FN, 'K': 3}, 'train_args': {'epochs': 200, 'epochs_h': 25, 'epochs_W': 25, 'lr': .005, 'wd': .001},
     'leg': 'AFGNN'},

    {'arch': GFGCN, 'arc_args': {'diff_layer': GFGCNLayer, 'hid_dim': 50, 'n_layers': 3, 'K': 2, 'act': ACT, 'h0': 1,
     'l_act': LAST_ACT, 'dropout': .25,}, 'sel_GSO': {'gso_type': 'A', 'norm': True}, 'model': GF_NodeClassModel,
     'mod_args': {'loss': LOSS_FN, 'K': 2}, 'train_args': {'epochs': 200, 'epochs_h': 10, 'epochs_W': 10, 'lr': .005, 'wd': .001},
     'leg': 'AFGNN-normA'},

    {'arch': GFGCN_Spows, 'arc_args': {'hid_dim': 32, 'n_layers': 3, 'K': 2, 'act': ACT, 'dev': device,
     'l_act': LAST_ACT, 'dropout': .5,}, 'sel_GSO': {'gso_type': 'A', 'norm': True}, 'model': GF_NodeClassModel,
     'mod_args': {'loss': LOSS_FN, 'K': 2}, 'train_args': {'epochs': 200, 'epochs_h': 5, 'epochs_W': 25, 'lr': .005, 'wd': 5e-4},
     'leg': 'AFGNN-normH'},
     
    ### Baselines
    {'arch': GFGCN, 'arc_args': {'diff_layer': GFGCN_noh_Layer, 'hid_dim': 32, 'n_layers': 2, 'K': 2, 'act': ACT, 
     'l_act': LAST_ACT, 'dropout': .25,}, 'sel_GSO': {'gso_type': 'A', 'norm': False}, 'model': NodeClassModel,
     'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 500, 'lr': .005, 'wd': .01}, 'leg': 'W-GCN'},

    {'arch': GFGCN, 'arc_args': {'diff_layer': GFGCN_noh_Layer, 'hid_dim': 32, 'n_layers': 2, 'K': 2, 'act': ACT, 
     'l_act': LAST_ACT, 'dropout': .25,}, 'sel_GSO': {'gso_type': 'A', 'norm': True}, 'model': NodeClassModel,
     'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 500, 'lr': .005, 'wd': .01}, 'leg': 'W-GCN-norm'},

    # GCNNs
    {'arch': GCNN, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'norm': 'both'}, 'mod_args': {'loss': LOSS_FN},
     'sel_GSO': {'gso_type': 'A-dgl'}, 'model': NodeClassModel, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'Kipf-norm'},

    {'arch': GCNN, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': nn.Identity(), 'dropout': .5, 'norm': 'both'}, 'mod_args': {'loss': LOSS_FN},
     'sel_GSO': {'gso_type': 'A-dgl'}, 'model': NodeClassModel, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'Kipf-norm-Id'},

    {'arch': GCNN, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'norm': 'none'}, 'sel_GSO': {'gso_type': 'A-dgl'},
     'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'Kipf'},
    # MLP
    {'arch': MLP, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'n_layers': 2}, 'sel_GSO': {'gso_type': 'none'},
     'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'MLP'},

    {'arch': MLP, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': nn.Identity(), 'dropout': .5, 'n_layers': 2}, 'sel_GSO': {'gso_type': 'none'},
     'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'MLP-Id'},

    
    # GAT
    {'arch': GAT, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'num_heads': 2, 'gat_params': {'attn_drop': 0}},
     'sel_GSO': {'gso_type': 'A-dgl', 'self_loops': False}, 'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200,
     'lr': .01, 'wd': 5e-4}, 'leg': 'GAT'},
    
    {'arch': GraphSAGE, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'n_layers': 2,},
     'sel_GSO': {'gso_type': 'A-dgl', 'self_loops': False}, 'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4},
     'leg': 'SAGE'},
    
    {'arch': GIN, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'n_layers': 2, 'mlp_layers': 2},
     'sel_GSO': {'gso_type': 'A-dgl', 'self_loops': False}, 'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4},
     'leg': 'GIN'},
]


accs_test_het, accs_max_val_het, ellapsed_times_het = run_exp(EXPS, DATASETS, N_RUNS, device)

table_accs_het = utils.summary_table(accs_test_het, DATASETS, EXPS)
table_accs_maxval_het = utils.summary_table(accs_max_val_het, DATASETS, EXPS)
table_time_het= utils.summary_table(ellapsed_times_het, DATASETS, EXPS)

TexasDataset
0: 0.811 (0.255) - 0.784 (0.162) - 0.865 (0.357) - 0.784 (0.020) - 0.838 (0.010) - 0.622 (0.017) - 0.622 (0.016) - 0.568 (0.017) - 0.811 (0.007) - 0.784 (0.004) - 0.730 (0.030) - 0.919 (0.021) - 0.649 (0.010)
1: 0.919 (0.339) - 0.946 (0.161) - 1.000 (0.363) - 0.811 (0.020) - 0.811 (0.013) - 0.595 (0.014) - 0.595 (0.015) - 0.730 (0.015) - 0.919 (0.007) - 0.757 (0.005) - 0.730 (0.039) - 0.811 (0.021) - 0.649 (0.020)
2: 0.838 (0.358) - 0.865 (0.166) - 0.865 (0.352) - 0.676 (0.014) - 0.757 (0.018) - 0.514 (0.018) - 0.514 (0.017) - 0.324 (0.013) - 0.811 (0.008) - 0.703 (0.005) - 0.486 (0.022) - 0.703 (0.022) - 0.486 (0.017)
3: 0.811 (0.359) - 0.919 (0.164) - 0.919 (0.329) - 0.703 (0.014) - 0.784 (0.015) - 0.595 (0.015) - 0.622 (0.015) - 0.622 (0.018) - 0.838 (0.008) - 0.838 (0.005) - 0.649 (0.022) - 0.757 (0.021) - 0.595 (0.010)
4: 0.838 (0.352) - 0.811 (0.153) - 0.838 (0.357) - 0.811 (0.018) - 0.838 (0.011) - 0.541 (0.016) - 0.514 (0.017) - 0.568 (0.018) - 0.838 (0.008) - 0.81

In [4]:
if SAVE:
    timestr = time.strftime("%Y%m%d-%H%M")
    table_accs_het.to_csv(PATH + 'heterophilic' + timestr)

leg = [exp['leg'] for exp in EXPS]

utils.display_all_data(leg, DATASETS, accs_test_het, agg='mean')
utils.display_all_data(leg, DATASETS, accs_test_het, agg='median')

,Exp,Texas (0.11),Wisconsin (0.20),Cornell (0.13),Chameleon (0.24)
0,AFGNN,0.827 ± 0.05013,0.8706 ± 0.02511,0.7973 ± 0.05701,0.4419 ± 0.01504
1,AFGNN-normA,0.8405 ± 0.05331,0.8549 ± 0.02935,0.7784 ± 0.04324,0.4274 ± 0.02623
2,AFGNN-normH,0.8757 ± 0.05156,0.849 ± 0.03513,0.7649 ± 0.04842,0.466 ± 0.02464
3,W-GCN,0.7568 ± 0.04187,0.7745 ± 0.0498,0.7216 ± 0.04531,0.4235 ± 0.0232
4,W-GCN-norm,0.8081 ± 0.0409,0.8549 ± 0.0266,0.7351 ± 0.04804,0.4406 ± 0.01928
5,Kipf-norm,0.5784 ± 0.04045,0.5333 ± 0.04706,0.4595 ± 0.09894,0.6268 ± 0.02108
6,Kipf-norm-Id,0.5757 ± 0.04531,0.5294 ± 0.06444,0.4189 ± 0.06074,0.6239 ± 0.01848
7,Kipf,0.5703 ± 0.1007,0.498 ± 0.09453,0.4405 ± 0.1012,0.2789 ± 0.02965
8,MLP,0.8135 ± 0.04595,0.849 ± 0.04476,0.7541 ± 0.0307,0.4763 ± 0.01765
9,MLP-Id,0.773 ± 0.04222,0.8314 ± 0.04314,0.7243 ± 0.04804,0.4851 ± 0.03528


,Exp,Texas (0.11),Wisconsin (0.20),Cornell (0.13),Chameleon (0.24)
0,AFGNN,0.8243 ± 0.05013,0.8725 ± 0.02511,0.8243 ± 0.05701,0.4397 ± 0.01504
1,AFGNN-normA,0.8108 ± 0.05331,0.8627 ± 0.02935,0.7703 ± 0.04324,0.4167 ± 0.02623
2,AFGNN-normH,0.8649 ± 0.05156,0.8431 ± 0.03513,0.7838 ± 0.04842,0.4671 ± 0.02464
3,W-GCN,0.7568 ± 0.04187,0.7549 ± 0.0498,0.7162 ± 0.04531,0.4331 ± 0.0232
4,W-GCN-norm,0.8243 ± 0.0409,0.8431 ± 0.0266,0.7432 ± 0.04804,0.4386 ± 0.01928
5,Kipf-norm,0.5946 ± 0.04045,0.5392 ± 0.04706,0.4459 ± 0.09894,0.636 ± 0.02108
6,Kipf-norm-Id,0.5811 ± 0.04531,0.5392 ± 0.06444,0.4189 ± 0.06074,0.6283 ± 0.01848
7,Kipf,0.5811 ± 0.1007,0.5294 ± 0.09453,0.4459 ± 0.1012,0.2895 ± 0.02965
8,MLP,0.8108 ± 0.04595,0.8235 ± 0.04476,0.7568 ± 0.0307,0.477 ± 0.01765
9,MLP-Id,0.7703 ± 0.04222,0.8333 ± 0.04314,0.7162 ± 0.04804,0.489 ± 0.03528


## Homophilicc datasets

In [5]:
DATASETS = ['CoraGraphDataset', 'CiteseerGraphDataset']
ACT = nn.ReLU()
LAST_ACT = nn.Identity()
LOSS_FN = nn.CrossEntropyLoss()

EXPS = [
    ### Our models
    {'arch': GFGCN, 'arc_args': {'diff_layer': GFGCNLayer, 'hid_dim': 64, 'n_layers': 2, 'K': 3, 'act': ACT, 'h0': 1,
     'l_act': nn.LogSoftmax(dim=1), 'dropout': .25,}, 'sel_GSO': {'gso_type': 'A', 'norm': False}, 'model': GF_NodeClassModel,
     'mod_args': {'loss': LOSS_FN, 'K': 3}, 'train_args': {'epochs': 200, 'epochs_h': 25, 'epochs_W': 25, 'lr': .01, 'wd': .001},
     'leg': 'AFGNN'},

    {'arch': GFGCN, 'arc_args': {'diff_layer': GFGCNLayer, 'hid_dim': 64, 'n_layers': 2, 'K': 3, 'act': ACT, 'h0': 1,
     'l_act': nn.LogSoftmax(dim=1), 'dropout': .25,}, 'sel_GSO': {'gso_type': 'A', 'norm': True}, 'model': GF_NodeClassModel,
     'mod_args': {'loss': LOSS_FN, 'K': 3}, 'train_args': {'epochs': 200, 'epochs_h': 25, 'epochs_W': 25, 'lr': .01, 'wd': .001},
     'leg': 'AFGNN-normA'},

    {'arch': GFGCN_Spows, 'arc_args': {'hid_dim': 32, 'n_layers': 3, 'K': 2, 'act': ACT, 'dev': device,
     'l_act': LAST_ACT, 'dropout': .5,}, 'sel_GSO': {'gso_type': 'A', 'norm': True}, 'model': GF_NodeClassModel,
     'mod_args': {'loss': LOSS_FN, 'K': 2}, 'train_args': {'epochs': 200, 'epochs_h': 5, 'epochs_W': 25, 'lr': .001, 'wd': 5e-4},
     'leg': 'AFGNN-normH'},

    # ### Baselines
    {'arch': GFGCN, 'arc_args': {'diff_layer': GFGCN_noh_Layer, 'hid_dim': 64, 'n_layers': 2, 'K': 3, 'act': nn.Tanh(), 
     'l_act': LAST_ACT, 'dropout': .25,}, 'sel_GSO': {'gso_type': 'A', 'norm': False}, 'model': NodeClassModel,
     'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 500, 'lr': .01, 'wd': .01}, 'leg': 'W-GCN'},

    {'arch': GFGCN, 'arc_args': {'diff_layer': GFGCN_noh_Layer, 'hid_dim': 64, 'n_layers': 2, 'K': 3, 'act': nn.Tanh(), 
     'l_act': LAST_ACT, 'dropout': .25,}, 'sel_GSO': {'gso_type': 'A', 'norm': True}, 'model': NodeClassModel,
     'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 500, 'lr': .01, 'wd': .01}, 'leg': 'W-GCN-norm'},

    # # GCNNs
    {'arch': GCNN, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'norm': 'both'}, 'mod_args': {'loss': LOSS_FN},
     'sel_GSO': {'gso_type': 'A-dgl'}, 'model': NodeClassModel, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'Kipf-norm'},

    {'arch': GCNN, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': nn.Identity(), 'dropout': .5, 'norm': 'both'}, 'mod_args': {'loss': LOSS_FN},
     'sel_GSO': {'gso_type': 'A-dgl'}, 'model': NodeClassModel, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'Kipf-norm-Id'},
    
    {'arch': GCNN, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'norm': 'none'}, 'sel_GSO': {'gso_type': 'A-dgl'},
     'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'Kipf'},
    # MLP
    {'arch': MLP, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'n_layers': 3}, 'sel_GSO': {'gso_type': 'none'},
     'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'MLP'},
    
    {'arch': MLP, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': nn.Identity(), 'dropout': .5, 'n_layers': 3}, 'sel_GSO': {'gso_type': 'none'},
     'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4}, 'leg': 'MLP-Id'},
    
    # GAT
    {'arch': GAT, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'num_heads': 2, 'gat_params': {'attn_drop': 0}},
     'sel_GSO': {'gso_type': 'A-dgl', 'self_loops': False}, 'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200,
     'lr': .01, 'wd': 5e-4}, 'leg': 'GAT'},
    
    {'arch': GraphSAGE, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'n_layers': 2,},
     'sel_GSO': {'gso_type': 'A-dgl', 'self_loops': False}, 'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4},
     'leg': 'SAGE'},
    
    {'arch': GIN, 'arc_args': {'hid_dim': 16, 'act': ACT, 'l_act': LAST_ACT, 'dropout': .5, 'n_layers': 2, 'mlp_layers': 2},
     'sel_GSO': {'gso_type': 'A-dgl', 'self_loops': False}, 'model': NodeClassModel, 'mod_args': {'loss': LOSS_FN}, 'train_args': {'epochs': 200, 'lr': .01, 'wd': 5e-4},
     'leg': 'GIN'},
]


accs_test_hom, accs_max_val_hom, ellapsed_times_hom = run_exp(EXPS, DATASETS, N_RUNS, device)

table_accs_hom = utils.summary_table(accs_test_hom, DATASETS, EXPS)
table_accs_maxval_hom = utils.summary_table(accs_max_val_hom, DATASETS, EXPS)
table_time_hom = utils.summary_table(ellapsed_times_hom, DATASETS, EXPS)

CoraGraphDataset
0: 0.701 (0.358) - 0.802 (0.405) - 0.792 (1.157) - 0.727 (0.117) - 0.807 (0.059) - 0.814 (0.029) - 0.808 (0.029) - 0.763 (0.010) - 0.366 (0.007) - 0.478 (0.008) - 0.769 (0.035) - 0.801 (0.022) - 0.755 (0.013)
1: 0.700 (0.407) - 0.800 (0.406) - 0.792 (1.730) - 0.616 (0.044) - 0.807 (0.058) - 0.811 (0.034) - 0.805 (0.033) - 0.749 (0.011) - 0.335 (0.008) - 0.441 (0.008) - 0.755 (0.043) - 0.800 (0.022) - 0.736 (0.013)
2: 0.737 (0.403) - 0.803 (0.398) - 0.781 (1.106) - 0.724 (0.057) - 0.802 (0.058) - 0.803 (0.029) - 0.811 (0.029) - 0.755 (0.010) - 0.455 (0.008) - 0.470 (0.008) - 0.755 (0.033) - 0.802 (0.022) - 0.697 (0.013)
3: 0.756 (0.410) - 0.807 (0.403) - 0.799 (1.839) - 0.546 (0.044) - 0.809 (0.057) - 0.805 (0.034) - 0.803 (0.031) - 0.761 (0.011) - 0.458 (0.008) - 0.400 (0.007) - 0.768 (0.034) - 0.814 (0.022) - 0.772 (0.013)
4: 0.703 (0.292) - 0.798 (0.277) - 0.785 (1.196) - 0.717 (0.092) - 0.809 (0.058) - 0.809 (0.034) - 0.808 (0.034) - 0.747 (0.011) - 0.335 (0.007) - 

In [6]:
if SAVE:
    timestr = time.strftime("%Y%m%d-%H%M")
    table_accs_hom.to_csv(PATH + 'homophilic' + timestr)

leg = [exp['leg'] for exp in EXPS]
utils.display_all_data(leg, DATASETS, accs_test_hom, agg='mean')
utils.display_all_data(leg, DATASETS, accs_test_hom, agg='median')


,Exp,CoraGraph (0.81),CiteseerGraph (0.74)
0,AFGNN,0.7262 ± 0.0203,0.6246 ± 0.02526
1,AFGNN-normA,0.8027 ± 0.004173,0.7006 ± 0.008845
2,AFGNN-normH,0.791 ± 0.009187,0.6942 ± 0.01283
3,W-GCN,0.6784 ± 0.05817,0.5429 ± 0.05956
4,W-GCN-norm,0.8062 ± 0.002926,0.7005 ± 0.007032
5,Kipf-norm,0.8077 ± 0.003716,0.7094 ± 0.003852
6,Kipf-norm-Id,0.8078 ± 0.005845,0.7104 ± 0.003007
7,Kipf,0.7539 ± 0.01052,0.6441 ± 0.009354
8,MLP,0.4215 ± 0.06809,0.4168 ± 0.06632
9,MLP-Id,0.4523 ± 0.04055,0.4255 ± 0.04805


,Exp,CoraGraph (0.81),CiteseerGraph (0.74)
0,AFGNN,0.7275 ± 0.0203,0.621 ± 0.02526
1,AFGNN-normA,0.8025 ± 0.004173,0.701 ± 0.008845
2,AFGNN-normH,0.792 ± 0.009187,0.695 ± 0.01283
3,W-GCN,0.713 ± 0.05817,0.5705 ± 0.05956
4,W-GCN-norm,0.807 ± 0.002926,0.7 ± 0.007032
5,Kipf-norm,0.808 ± 0.003716,0.7105 ± 0.003852
6,Kipf-norm-Id,0.808 ± 0.005845,0.7105 ± 0.003007
7,Kipf,0.755 ± 0.01052,0.641 ± 0.009354
8,MLP,0.4565 ± 0.06809,0.433 ± 0.06632
9,MLP-Id,0.455 ± 0.04055,0.43 ± 0.04805
